In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('all_df_var.csv')

In [3]:
df = df[df.columns[1:]]
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,Merchnum_sum_1_30,Merchnum_sum_2_7,Merchnum_sum_2_14,Merchnum_sum_2_30,MerchDesc_sum_1_7,MerchDesc_sum_1_14,MerchDesc_sum_1_30,MerchDesc_sum_2_7,MerchDesc_sum_2_14,MerchDesc_sum_2_30
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,...,0.077264,1.645279,0.079154,0.077264,3.683257,0.439841,1.319523,3.683257,0.439841,1.319523
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,...,3.000000,7.000000,1.000000,3.000000,7.000000,1.000000,3.000000,7.000000,1.000000,3.000000
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,...,0.559881,7.000000,0.224146,0.559881,7.000000,0.224146,0.559881,7.000000,0.224146,0.559881
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,...,0.077264,1.645279,0.079154,0.077264,2.179301,0.256098,0.768293,2.179301,0.256098,0.768293
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,...,0.077264,1.645279,0.079154,0.077264,3.683257,0.439841,1.319523,3.683257,0.439841,1.319523


In [4]:
df = df[['Recnum', 'Cardnum', 'Merchnum', 'Merch description', 'Amount']]

In [5]:
idx = []
for i in range(len(df)):
    if 'FEDEX' in df.loc[i, 'Merch description']:
        idx.append(df.loc[i, 'Recnum'])

In [6]:
df = df.set_index(df.Recnum)

In [7]:
df = df.drop(labels=idx, axis=0)

In [8]:
df.loc[:,'Amount'] = df.loc[:,'Amount'].astype('str')

In [9]:
df.loc[:,'Amount_digit'] = [df.loc[i, 'Amount'][0] for i in df.index]

In [10]:
df_card = df[(df['Amount_digit'] == '1') | (df['Amount_digit'] == '2')]

df_low = df_card.groupby(['Cardnum'])['Recnum'].count().reset_index().rename(columns = {'Recnum':'n_low'})
df_card_sum = df.groupby('Cardnum')['Recnum'].count().reset_index().rename(columns = {'Recnum':'n'})
df_card_sum = df_card_sum.merge(df_low, how='left', on='Cardnum')
df_card_sum['n_low'] = df_card_sum['n_low'].fillna(0)
df_card_sum['n_high'] = df_card_sum['n'] - df_card_sum['n_low']

df_card_sum['n_low'] = df_card_sum['n_low'].replace(0,1)
df_card_sum['n_high'] = df_card_sum['n_high'].replace(0,1)

In [11]:
df_merc = df[(df['Amount_digit'] == '1') | (df['Amount_digit'] == '2')]
df_low_m = df_merc.groupby(['Merchnum'])['Recnum'].count().reset_index().rename(columns = {'Recnum':'n_low'})
df_sum_m = df.groupby(['Merchnum'])['Recnum'].count().reset_index().rename(columns = {'Recnum':'n'})
df_sum_m = df_sum_m.merge(df_low_m, how='left', on='Merchnum')

df_sum_m['n_low'] = df_sum_m['n_low'].fillna(0)
df_sum_m['n_high'] = df_sum_m['n'] - df_sum_m['n_low']
df_sum_m['n_low'] = df_sum_m['n_low'].replace(0,1)
df_sum_m['n_high'] = df_sum_m['n_high'].replace(0,1)


In [12]:
df_card_sum['R'] = (1.096 * df_card_sum['n_low'])/df_card_sum['n_high']
df_sum_m['R'] = (1.096 * df_sum_m['n_low'])/df_sum_m['n_high']

In [13]:
df_card_sum['1/R'] = 1/df_card_sum['R']
df_sum_m['1/R'] = 1/df_sum_m['R']

In [14]:
df_card_sum.loc[:,'U'] = [max(df_card_sum.loc[i,'R'], df_card_sum.loc[i,'1/R']) for i in df_card_sum.index]
df_sum_m.loc[:,'U'] = [max(df_sum_m.loc[i,'R'], df_sum_m.loc[i,'1/R']) for i in df_sum_m.index]



In [15]:
df_card_sum['t'] = (df_card_sum['n']-15)/3
df_sum_m['t'] = (df_sum_m['n']-15)/3

In [16]:
df_card_sum['U*']  = 1 + ((df_card_sum['U'] - 1)/ (1 + np.exp(-df_card_sum['t'])))
df_sum_m['U*']  = 1 + ((df_sum_m['U'] - 1)/ (1 + np.exp(-df_sum_m['t'])))

In [17]:
df_card_sum = df_card_sum.sort_values('U*', ascending = False)
df_card_sum = df_card_sum[df_card_sum['U*'] !=float("inf")]

In [18]:
df_sum_m = df_sum_m.sort_values('U*', ascending = False)
df_sum_m = df_sum_m[df_sum_m['U*'] !=float("inf")]

In [26]:
card_final = df_card_sum.head(40)
card_final.round(2).to_csv('card_benford.csv', index=False)

In [29]:
merc_final = df_sum_m.head(40)
merc_final.round(2).to_csv('merc_benford.csv', index=False)

In [30]:
merc_final

,Merchnum,n,n_low,n_high,R,1/R,U,t,U*
12841,991808369338,181,1.0,181.0,0.006055,165.145985,165.145985,55.333333,165.145985
9819,8078200641472,60,59.0,1.0,64.664000,0.015465,64.664000,15.000000,64.663981
1948,308904389335,53,1.0,53.0,0.020679,48.357664,48.357664,12.666667,48.357515
2347,3523000628102,34,34.0,1.0,37.264000,0.026836,37.264000,6.333333,37.199706
9913,808998385332,37,1.0,36.0,0.030444,32.846715,32.846715,7.333333,32.825921
4979,55158027,28,27.0,1.0,29.592000,0.033793,29.592000,4.333333,29.221627
10739,8916500620062,31,1.0,31.0,0.035355,28.284672,28.284672,5.333333,28.153575
2602,3910694900001,26,25.0,1.0,27.400000,0.036496,27.400000,3.666667,26.741995
10661,8889817332,25,24.0,1.0,26.304000,0.038017,26.304000,3.333333,25.432399
10515,881145544,24,24.0,1.0,26.304000,0.038017,26.304000,3.000000,25.103936
